# 03-SQL-Consultas-Avanzadas

In [1]:
# Librerias
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Conerctar a mi SQLserver
driver = "ODBC Driver 17 for SQL Server"
server = r"DESKTOP-GB9FFBV"
database = "BD_SQL_"

In [3]:
# Crear motor de conexión
connection_string = f"mssql+pyodbc://@{server}/{database}?driver={driver}"
engine = create_engine(connection_string)

In [4]:
# -------------------------------
# 1. Calcular la variación porcentual de ventas mes a mes por producto.
# -------------------------------
df1 = pd.read_sql("""
WITH t1 AS (
    SELECT v.id_producto, p.nombre_producto,
           DATEPART(year, fecha_venta) AS anio,
           DATEPART(month, fecha_venta) AS mes,
           SUM(v.precio_unitario * v.cantidad) AS total_venta,
           LAG(SUM(v.precio_unitario * v.cantidad),1) OVER (PARTITION BY v.id_producto ORDER BY DATEPART(year, fecha_venta)) AS total_venta_anterior
    FROM ventas v INNER JOIN productos p ON v.id_producto = p.id_producto
    GROUP BY v.id_producto, p.nombre_producto, DATEPART(year, fecha_venta), DATEPART(month, fecha_venta)
)
SELECT id_producto, nombre_producto, anio, mes,
       (total_venta - total_venta_anterior) / total_venta_anterior * 100 AS variacion
FROM t1;
""", engine)
print(df1)

      id_producto nombre_producto  anio  mes  variacion
0               1      Producto 1  2023    7        NaN
1               1      Producto 1  2023    1  35.463885
2               1      Producto 1  2023   10  -0.499514
3               1      Producto 1  2023    4  29.962401
4               1      Producto 1  2023    5 -25.986011
...           ...             ...   ...  ...        ...
1235           40     Producto 40  2025    1   4.700500
1236           40     Producto 40  2025    6  -3.121225
1237           40     Producto 40  2025    4  -7.262485
1238           40     Producto 40  2025    3  45.922950
1239           40     Producto 40  2025    2  17.773857

[1240 rows x 5 columns]


In [5]:
# -------------------------------
# 2. Identificar clientes con tres meses consecutivos de crecimiento en compras.
# -------------------------------
df2 = pd.read_sql("""
WITH t1 AS (
    SELECT v.id_cliente, c.nombre,
           DATEPART(year, v.fecha_venta) AS anio,
           DATEPART(month, v.fecha_venta) AS mes,
           SUM(v.precio_unitario * v.cantidad) AS total_mes,
           LAG(SUM(v.precio_unitario * v.cantidad),1) OVER (PARTITION BY v.id_cliente ORDER BY DATEPART(year, v.fecha_venta), DATEPART(month, v.fecha_venta)) AS mes_anterior,
           LAG(SUM(v.precio_unitario * v.cantidad),2) OVER (PARTITION BY v.id_cliente ORDER BY DATEPART(year, v.fecha_venta), DATEPART(month, v.fecha_venta)) AS mes_trasanterior
    FROM ventas v INNER JOIN clientes c ON v.id_cliente = c.id_cliente
    GROUP BY v.id_cliente, c.nombre, DATEPART(year, v.fecha_venta), DATEPART(month, v.fecha_venta)
),
t2 AS (
    SELECT id_cliente, nombre, anio, mes, total_mes, mes_anterior, mes_trasanterior,
           CASE WHEN total_mes > mes_anterior AND mes_anterior > mes_trasanterior THEN 1 ELSE 0 END AS MesesCrecimiento
    FROM t1
)
SELECT id_cliente, nombre, anio, mes, total_mes, mes_anterior, mes_trasanterior
FROM t2
WHERE MesesCrecimiento = 0;
""", engine)
print(df2)

       id_cliente       nombre  anio  mes    total_mes  mes_anterior  \
0               1    Cliente 1  2023    1  1056.089981           NaN   
1               1    Cliente 1  2023    2  1730.719971   1056.089981   
2               1    Cliente 1  2023    3  1694.730011   1730.719971   
3               1    Cliente 1  2023    5  2042.939941   1694.730011   
4               1    Cliente 1  2023    6   859.749985   2042.939941   
...           ...          ...   ...  ...          ...           ...   
15457         800  Cliente 800  2024    9   331.510010    552.239990   
15458         800  Cliente 800  2024   10   374.620018    331.510010   
15459         800  Cliente 800  2025    4   215.039993   1352.609997   
15460         800  Cliente 800  2025    5   124.000000    215.039993   
15461         800  Cliente 800  2025    6   882.100006    124.000000   

       mes_trasanterior  
0                   NaN  
1                   NaN  
2           1056.089981  
3           1730.719971  
4    

In [6]:
# -------------------------------
# 3. Calcular la mediana de salarios por departamento.
# -------------------------------
df3 = pd.read_sql("""
SELECT DISTINCT d.id_departamento, d.nombre_departamento,
       PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY e.salario) OVER (PARTITION BY d.id_departamento) AS media
FROM departamentos d LEFT JOIN empleados e ON d.id_departamento = e.id_departamento;
""", engine)
print(df3)

    id_departamento nombre_departamento        media
0                 1             Depto 1  5875.004883
1                 2             Depto 2  2221.429932
2                 3             Depto 3  6011.979980
3                 4             Depto 4  3484.520020
4                 5             Depto 5  5457.135010
5                 6             Depto 6  6882.070068
6                 7             Depto 7          NaN
7                 8             Depto 8  6868.859863
8                 9             Depto 9  4806.419922
9                10            Depto 10          NaN
10               11            Depto 11  4853.114990
11               12            Depto 12  6969.410156
12               13            Depto 13  3613.179932
13               14            Depto 14          NaN
14               15            Depto 15          NaN
15               16            Depto 16  7864.180176
16               17            Depto 17          NaN
17               18            Depto 18       

In [7]:
# -------------------------------
# 4. Mostrar los productos con ventas crecientes en los últimos tres periodos.
# -------------------------------
df4 = pd.read_sql("""
WITH t1 AS (
    SELECT p.id_producto, p.nombre_producto,
           DATETRUNC(month, v.fecha_venta) AS mes,
           SUM(v.precio_unitario * v.cantidad) AS t_mes_base,
           LAG(SUM(v.precio_unitario * v.cantidad), 1) OVER (PARTITION BY p.id_producto ORDER BY DATETRUNC(month, v.fecha_venta)) AS t_mes_sig,
           LAG(SUM(v.precio_unitario * v.cantidad), 2) OVER (PARTITION BY p.id_producto ORDER BY DATETRUNC(month, v.fecha_venta)) AS t_mes_2sig
    FROM ventas v INNER JOIN productos p ON v.id_producto = p.id_producto
    GROUP BY p.id_producto, p.nombre_producto, DATETRUNC(month, v.fecha_venta)
),
t2 AS (
    SELECT *, CASE WHEN t_mes_base > t_mes_sig AND t_mes_sig > t_mes_2sig THEN 1 ELSE 0 END AS Cumple
    FROM t1
)
SELECT * FROM t2 WHERE Cumple = 1;
""", engine)
print(df4)

     id_producto nombre_producto         mes    t_mes_base     t_mes_sig  \
0              1      Producto 1  2024-05-01  33388.979761  22072.540009   
1              1      Producto 1  2024-11-01  29605.669952  17562.709871   
2              1      Producto 1  2025-06-01  16315.829979  15966.879944   
3              1      Producto 1  2025-07-01  23002.639740  16315.829979   
4              2      Producto 2  2023-03-01  28110.659817  23935.489914   
..           ...             ...         ...           ...           ...   
189           40     Producto 40  2024-01-01  25260.829891  22062.350224   
190           40     Producto 40  2024-04-01  24632.640112  20666.699909   
191           40     Producto 40  2024-05-01  25577.830158  24632.640112   
192           40     Producto 40  2025-02-01  32559.889736  21087.560078   
193           40     Producto 40  2025-06-01  20429.369905  20140.839878   

       t_mes_2sig  Cumple  
0    16145.449932       1  
1    16381.479988       1  
2  

In [8]:
# -------------------------------
# 5. Obtener el top 2 de vendedores por región usando funciones analíticas.
# -------------------------------
df5 = pd.read_sql("""
WITH t1 AS (
    SELECT v.region, e.id_vendedor, e.nombre,
           SUM(v.precio_unitario * cantidad) AS total,
           DENSE_RANK() OVER (PARTITION BY v.region ORDER BY SUM(v.precio_unitario * cantidad)) AS ranking
    FROM ventas v INNER JOIN empleados e ON v.id_vendedor = e.id_vendedor
    GROUP BY v.region, e.id_vendedor, e.nombre
)
SELECT * FROM t1 WHERE ranking < 2;
""", engine)
print(df5)

   region  id_vendedor       nombre          total  ranking
0  Centro           12  Empleado 12  145772.980022        1
1   Norte           24  Empleado 24  138353.859585        1
2     Sur           28  Empleado 28  138715.470005        1


In [9]:
# -------------------------------
# 6. Detectar empleados que nunca tuvieron una reducción salarial.
# -------------------------------
print("Consulta no ejecutable: faltan columnas históricas de salario para resolver esta pregunta.")

Consulta no ejecutable: faltan columnas históricas de salario para resolver esta pregunta.


In [10]:
# -------------------------------
# 7. Calcular el acumulado de ventas (running total) por cliente.
# -------------------------------
df7 = pd.read_sql("""
WITH t1 AS (
    SELECT DATETRUNC(month, v.fecha_venta) AS periodo,
           v.id_cliente, c.nombre,
           SUM(v.precio_unitario * v.cantidad) AS total
    FROM ventas v INNER JOIN clientes c ON v.id_cliente = c.id_cliente
    GROUP BY DATETRUNC(month, v.fecha_venta), v.id_cliente, c.nombre
)
SELECT *, SUM(total) OVER (PARTITION BY id_cliente ORDER BY periodo) AS acumulado
FROM t1;
""", engine)
print(df7)

          periodo  id_cliente       nombre        total     acumulado
0      2023-01-01           1    Cliente 1  1056.089981   1056.089981
1      2023-02-01           1    Cliente 1  1730.719971   2786.809952
2      2023-03-01           1    Cliente 1  1694.730011   4481.539963
3      2023-05-01           1    Cliente 1  2042.939941   6524.479904
4      2023-06-01           1    Cliente 1   859.749985   7384.229889
...           ...         ...          ...          ...           ...
18196  2025-02-01         800  Cliente 800  1352.609997  23423.579983
18197  2025-04-01         800  Cliente 800   215.039993  23638.619976
18198  2025-05-01         800  Cliente 800   124.000000  23762.619976
18199  2025-06-01         800  Cliente 800   882.100006  24644.719982
18200  2025-07-01         800  Cliente 800  1250.560059  25895.280041

[18201 rows x 5 columns]


In [11]:
# -------------------------------
# 8. Determinar el tiempo promedio entre compras por cliente.
# -------------------------------
df8 = pd.read_sql("""
WITH t1 AS (
    SELECT id_cliente, id_venta, fecha_venta,
           LAG(fecha_venta) OVER (PARTITION BY id_cliente ORDER BY fecha_venta) AS fecha_venta_anterior
    FROM ventas v
)
SELECT id_cliente, id_venta, fecha_venta, fecha_venta_anterior,
       AVG(DATEDIFF(day, fecha_venta_anterior, fecha_venta)) OVER (PARTITION BY id_cliente ORDER BY fecha_venta) AS prom_tiempo
FROM t1;
""", engine)
print(df8)

       id_cliente  id_venta fecha_venta fecha_venta_anterior  prom_tiempo
0               1     18172  2023-01-10                 None          NaN
1               1       188  2023-01-12           2023-01-10          2.0
2               1     15312  2023-02-05           2023-01-12         13.0
3               1     14137  2023-03-07           2023-02-05         18.0
4               1     29564  2023-03-10           2023-03-07         14.0
...           ...       ...         ...                  ...          ...
32762         800      9407  2025-04-21           2025-02-25         24.0
32763         800     26127  2025-05-13           2025-04-21         24.0
32764         800     11852  2025-06-06           2025-05-13         24.0
32765         800     27342  2025-06-26           2025-06-06         24.0
32766         800      6991  2025-07-25           2025-06-26         24.0

[32767 rows x 5 columns]


In [12]:
# -------------------------------
# 9. Identificar los 10 productos con mayor rentabilidad (ingresos - costo).
# -------------------------------
df9 = pd.read_sql("""
SELECT TOP 10 p.id_producto, p.nombre_producto,
       SUM(v.precio_unitario * cantidad) - SUM(p.precio_lista * v.cantidad) AS rentabilidad
FROM productos p INNER JOIN ventas v ON p.id_producto = v.id_producto
GROUP BY p.id_producto, p.nombre_producto;
""", engine)
print(df9)

   id_producto nombre_producto   rentabilidad
0           23     Producto 23  139390.936077
1           29     Producto 29  555878.270134
2           15     Producto 15 -498935.103289
3            9      Producto 9   62334.715202
4            3      Producto 3 -234497.495781
5           32     Producto 32  265664.530558
6           26     Producto 26  394417.102495
7           12     Producto 12 -116307.105106
8           35     Producto 35 -494823.397219
9            6      Producto 6   69626.572454


In [13]:
# -------------------------------
# 10. Calcular la distribución porcentual de ventas por canal y mes.
# -------------------------------
df10 = pd.read_sql("""
SELECT canal, DATETRUNC(month, fecha_venta) AS periodo,
       PERCENT_RANK() OVER (PARTITION BY canal ORDER BY SUM(precio_unitario * cantidad)) * 100 AS dist_porcentual
FROM ventas
GROUP BY canal, DATETRUNC(month, fecha_venta);
""", engine)
print(df10)

   canal     periodo  dist_porcentual
0    APP  2023-02-01         0.000000
1    APP  2023-01-01         3.333333
2    APP  2024-02-01         6.666667
3    APP  2025-04-01        10.000000
4    APP  2024-09-01        13.333333
..   ...         ...              ...
88   WEB  2024-08-01        86.666667
89   WEB  2023-03-01        90.000000
90   WEB  2023-07-01        93.333333
91   WEB  2024-09-01        96.666667
92   WEB  2024-12-01       100.000000

[93 rows x 3 columns]


In [14]:
# -------------------------------
# 11. Mostrar la tendencia de ventas trimestral (subida/bajada).
# -------------------------------
df11 = pd.read_sql("""
WITH t1 AS (
    SELECT DATEPART(year, fecha_venta) AS anio,
           DATEPART(quarter, fecha_venta) AS trimestre,
           SUM(precio_unitario * cantidad) AS total,
           LAG(SUM(precio_unitario * cantidad)) OVER (ORDER BY DATEPART(year, fecha_venta), DATEPART(quarter, fecha_venta)) AS trimestre_anterior
    FROM ventas
    GROUP BY DATEPART(year, fecha_venta), DATEPART(quarter, fecha_venta)
)
SELECT *, CASE WHEN total < trimestre_anterior THEN 'bajada'
               WHEN total > trimestre_anterior THEN 'subida'
               WHEN total = trimestre_anterior THEN 'sin_variacion'
               ELSE 'no_aplica' END AS indicador
FROM t1;
""", engine)
print(df11)

    anio  trimestre         total  trimestre_anterior  indicador
0   2023          1  2.378343e+06                 NaN  no_aplica
1   2023          2  2.627290e+06        2.378343e+06     subida
2   2023          3  2.423345e+06        2.627290e+06     bajada
3   2023          4  2.453568e+06        2.423345e+06     subida
4   2024          1  2.486619e+06        2.453568e+06     subida
5   2024          2  2.418425e+06        2.486619e+06     bajada
6   2024          3  2.520541e+06        2.418425e+06     subida
7   2024          4  2.547033e+06        2.520541e+06     subida
8   2025          1  2.485396e+06        2.547033e+06     bajada
9   2025          2  2.356042e+06        2.485396e+06     bajada
10  2025          3  8.230328e+05        2.356042e+06     bajada


In [16]:
# -------------------------------
# 12. Encontrar empleados cuyo salario está por encima del percentil 90.
# -------------------------------
df12 = pd.read_sql("""
WITH t1 AS (
    SELECT id_vendedor, nombre,
           PERCENT_RANK() OVER (ORDER BY salario) * 100 AS percentil
    FROM empleados
)
SELECT * FROM t1 WHERE percentil >= 90;
""", engine)
print(df12)

   id_vendedor       nombre   percentil
0           29  Empleado 29   91.836735
1           42  Empleado 42   93.877551
2           47  Empleado 47   95.918367
3           41  Empleado 41   97.959184
4           21  Empleado 21  100.000000


In [17]:
# -------------------------------
# 16. Identificar productos con ventas duplicadas en diferentes regiones.
# -------------------------------
df16 = pd.read_sql("""
SELECT v1.region, v1.id_producto, SUM(v1.cantidad) AS cantidad_1,
       v2.region, v2.id_producto, SUM(v2.cantidad) AS cantidad_2
FROM ventas v1 INNER JOIN ventas v2
  ON v1.region < v2.region AND v1.id_producto = v2.id_producto
GROUP BY v1.region, v2.region, v1.id_producto, v2.id_producto;
""", engine)
print(df16)

     region  id_producto  cantidad_1 region  id_producto  cantidad_2
0    Centro            9      241582  Norte            9      238497
1    Centro           12      210589  Norte           12      209440
2    Centro           15      203250  Norte           15      205820
3    Centro           18      205086  Norte           18      213332
4    Centro           38      266012  Norte           38      263993
..      ...          ...         ...    ...          ...         ...
115   Norte           11      213213    Sur           11      215204
116   Norte           14      214894    Sur           14      225780
117   Norte           17      219501    Sur           17      226486
118   Norte           20      237437    Sur           20      240219
119   Norte           40      234360    Sur           40      229425

[120 rows x 6 columns]


In [18]:
# -------------------------------
# 17. Calcular el promedio de ventas ponderado por cantidad.
# -------------------------------
df17 = pd.read_sql("SELECT SUM(precio_unitario * cantidad) / SUM(cantidad) AS ponderado FROM ventas;", engine)
print(df17)

    ponderado
0  259.356422


In [19]:
# -------------------------------
# 18. Mostrar el top 5% de vendedores según facturación total.
# -------------------------------
df18 = pd.read_sql("""
WITH t1 AS (
    SELECT v.id_vendedor, e.nombre,
           PERCENT_RANK() OVER (ORDER BY SUM(precio_unitario * cantidad)) AS ranking
    FROM ventas v INNER JOIN empleados e ON v.id_vendedor = e.id_vendedor
    GROUP BY v.id_vendedor, e.nombre
)
SELECT * FROM t1 WHERE ranking >= 0.95;
""", engine)
print(df18)

   id_vendedor       nombre   ranking
0           34  Empleado 34  0.959184
1           38  Empleado 38  0.979592
2           30  Empleado 30  1.000000


In [20]:
# -------------------------------
# 19. Detectar clientes cuya facturación está fuera del rango normal del global de compras.
# -------------------------------
df19 = pd.read_sql("""
WITH t1 AS (
    SELECT id_cliente,
           SUM(precio_unitario * cantidad) AS total_cliente,
           AVG(SUM(precio_unitario * cantidad)) OVER () AS promedio_global,
           STDEV(SUM(precio_unitario * cantidad)) OVER () AS desvia_global
    FROM ventas
    GROUP BY id_cliente
)
SELECT * FROM t1
WHERE total_cliente > promedio_global + 2 * desvia_global
   OR total_cliente < promedio_global - 2 * desvia_global;
""", engine)
print(df19)

    id_cliente  total_cliente  promedio_global  desvia_global
0          501   15934.619980      31899.54316    6365.970048
1          779   18696.580090      31899.54316    6365.970048
2          507   14399.370144      31899.54316    6365.970048
3          338   17566.680185      31899.54316    6365.970048
4          146   46654.009836      31899.54316    6365.970048
5          573   45303.130314      31899.54316    6365.970048
6          510   17937.200020      31899.54316    6365.970048
7          702   48907.379944      31899.54316    6365.970048
8          112   46614.010073      31899.54316    6365.970048
9           64   17152.939903      31899.54316    6365.970048
10         350   18952.909988      31899.54316    6365.970048
11         685   51230.529732      31899.54316    6365.970048
12         124   17318.490023      31899.54316    6365.970048
13         794   45216.379730      31899.54316    6365.970048
14         508   45661.790176      31899.54316    6365.970048
15      

In [21]:
# -------------------------------
# 20. Calcular la cantidad de días promedio entre pedido por región.
# -------------------------------
df20 = pd.read_sql("""
WITH t1 AS (
    SELECT region, fecha_venta,
           LEAD(fecha_venta,1) OVER (PARTITION BY region ORDER BY fecha_venta) AS fecha_siguiente,
           DATEDIFF(day, fecha_venta, LEAD(fecha_venta,1) OVER (PARTITION BY region ORDER BY fecha_venta)) AS dif_dias
    FROM ventas
)
SELECT region, AVG(dif_dias) AS promedio_días_region
FROM t1 GROUP BY region;
""", engine)
print(df20)

   region  promedio_días_region
0  Centro                     0
1   Norte                     0
2     Sur                     0


In [22]:
# -------------------------------
# 21. Obtener las 3 categorías con mayor aumento de ventas interanual.
# -------------------------------
df21 = pd.read_sql("""
WITH t1 AS (
    SELECT p.categoria, DATETRUNC(year, v.fecha_venta) AS periodo_anio,
           SUM(v.precio_unitario * v.cantidad) AS ventas_anuales,
           LAG(SUM(v.precio_unitario * v.cantidad), 1) OVER (PARTITION BY p.categoria ORDER BY DATETRUNC(year, v.fecha_venta)) AS ventas_anuales_anterior,
           SUM(v.precio_unitario * v.cantidad) - LAG(SUM(v.precio_unitario * v.cantidad), 1) OVER (PARTITION BY p.categoria ORDER BY DATETRUNC(year, v.fecha_venta)) AS variacion
    FROM ventas v INNER JOIN productos p ON v.id_producto = p.id_producto
    GROUP BY p.categoria, DATETRUNC(year, v.fecha_venta)
),
t2 AS (
    SELECT categoria, MAX(variacion) AS v_max
    FROM t1 GROUP BY categoria
)
SELECT TOP(3) t1.categoria, t1.periodo_anio, t1.ventas_anuales, t1.ventas_anuales_anterior, t1.variacion
FROM t1 INNER JOIN t2 ON t1.categoria = t2.categoria AND t1.variacion = t2.v_max
ORDER BY t1.variacion DESC;
""", engine)
print(df21)

  categoria periodo_anio  ventas_anuales  ventas_anuales_anterior  \
0  Juguetes   2024-01-01    3.309160e+06             3.118825e+06   
1     Hogar   2024-01-01    2.790910e+06             2.692195e+06   
2      Ropa   2024-01-01    2.236764e+06             2.303553e+06   

       variacion  
0  190334.928848  
1   98714.728830  
2  -66788.892008  


In [23]:
# -------------------------------
# 22. Analizar qué porcentaje de clientes generan el 80% de los ingresos (regla 80/20).
# -------------------------------
df22 = pd.read_sql("""
WITH t1 AS (
    SELECT id_cliente,
           SUM(precio_unitario * cantidad) AS venta,
           PERCENT_RANK() OVER (ORDER BY SUM(precio_unitario * cantidad)) * 100.00 AS incremento_ventas
    FROM ventas
    GROUP BY id_cliente
),
t2 AS (
    SELECT *, COUNT(id_cliente) OVER (ORDER BY incremento_ventas) * 1.0 /
                 COUNT(id_cliente) OVER () * 100 AS porcentaje_clientes
    FROM t1
)
SELECT venta, incremento_ventas, porcentaje_clientes
FROM t2 WHERE porcentaje_clientes <= 80;
""", engine)
print(df22)

            venta  incremento_ventas  porcentaje_clientes
0    14399.370144           0.000000                0.125
1    15934.619980           0.125156                0.250
2    16370.059978           0.250313                0.375
3    16467.619961           0.375469                0.500
4    17152.939903           0.500626                0.625
..            ...                ...                  ...
635  37281.579880          79.474343               79.500
636  37307.349796          79.599499               79.625
637  37349.689877          79.724656               79.750
638  37503.730263          79.849812               79.875
639  37551.050377          79.974969               80.000

[640 rows x 3 columns]


In [24]:
# -------------------------------
# 23. Encontrar el producto con mayor caída de ventas mes a mes.
# -------------------------------
df23 = pd.read_sql("""
WITH t1 AS (
    SELECT DATETRUNC(month, fecha_venta) AS periodo, id_producto,
           SUM(precio_unitario * cantidad) AS venta_mensual,
           LAG(SUM(precio_unitario * cantidad),1) OVER (PARTITION BY id_producto ORDER BY DATETRUNC(month, fecha_venta)) AS venta_mes_anterior
    FROM ventas
    GROUP BY DATETRUNC(month, fecha_venta), id_producto
),
t2 AS (
    SELECT periodo, MIN(venta_mes_anterior - venta_mensual) OVER (PARTITION BY periodo) AS menor_incremento
    FROM t1
)
SELECT a.periodo, a.id_producto, b.menor_incremento
FROM t1 a INNER JOIN t2 b ON a.periodo = b.periodo
  AND a.venta_mes_anterior - a.venta_mensual = b.menor_incremento
  AND a.venta_mes_anterior IS NOT NULL
GROUP BY a.periodo, a.id_producto, b.menor_incremento;
""", engine)
print(df23)

       periodo  id_producto  menor_incremento
0   2025-07-01           10     -12707.819885
1   2025-03-01            1     -17182.909870
2   2024-11-01           16     -12993.420073
3   2023-05-01            5     -18613.139904
4   2023-07-01           39     -14225.390133
5   2024-02-01           20     -13927.870094
6   2025-02-01           36     -18694.459984
7   2024-07-01            1     -16899.530045
8   2024-03-01           34     -17034.350101
9   2023-04-01           21     -28051.040070
10  2023-03-01           35     -15427.680149
11  2025-05-01            2     -12156.060213
12  2023-10-01            7     -17004.749786
13  2023-09-01           32     -13043.430149
14  2024-04-01           29     -18628.290371
15  2025-06-01            6     -12057.069946
16  2024-05-01            1     -11316.439753
17  2025-04-01           13     -13847.850142
18  2025-01-01           37     -16065.519987
19  2023-11-01           31     -13701.599977
20  2023-06-01            3     -1

In [25]:
# -------------------------------
# 24. Calcular la media móvil de 6 meses por categoría de producto.
# -------------------------------
df24 = pd.read_sql("""
SELECT p.categoria, DATETRUNC(month, v.fecha_venta) AS periodo,
       SUM(v.precio_unitario * v.cantidad) AS item1,
       AVG(v.precio_unitario * v.cantidad) AS promedio_de_importe_de_cada_mes,
       AVG(SUM(v.precio_unitario * v.cantidad)) OVER (PARTITION BY p.categoria ORDER BY DATETRUNC(month, v.fecha_venta) ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) AS promedio_mensual
FROM ventas v INNER JOIN productos p ON v.id_producto = p.id_producto
GROUP BY p.categoria, DATETRUNC(month, v.fecha_venta);
""", engine)
print(df24)

       categoria     periodo          item1  promedio_de_importe_de_cada_mes  \
0    Electronica  2023-01-01  155024.609795                       779.018140   
1    Electronica  2023-02-01  138642.420197                       761.771540   
2    Electronica  2023-03-01  143643.180031                       886.686296   
3    Electronica  2023-04-01  165020.380075                       789.571197   
4    Electronica  2023-05-01  171689.610134                       825.430818   
..           ...         ...            ...                              ...   
119         Ropa  2025-03-01  187035.229898                       763.409102   
120         Ropa  2025-04-01  171729.060129                       753.197632   
121         Ropa  2025-05-01  191949.119867                       830.948571   
122         Ropa  2025-06-01  171754.100172                       795.157871   
123         Ropa  2025-07-01  195374.699959                       807.333471   

     promedio_mensual  
0       155024.

In [26]:
# -------------------------------
# 25. Crear un ranking de clientes con base en su ticket promedio.
# -------------------------------
df25 = pd.read_sql("""
SELECT id_cliente,
       AVG(precio_unitario * cantidad) AS venta_promedio_cliente,
       DENSE_RANK() OVER (ORDER BY AVG(precio_unitario * cantidad) DESC) AS ranking
FROM ventas
GROUP BY id_cliente;
""", engine)
print(df25)

     id_cliente  venta_promedio_cliente  ranking
0           696             1077.421187        1
1           436             1058.055386        2
2           508             1037.767959        3
3           687             1032.201624        4
4           545             1029.016872        5
..          ...                     ...      ...
795         507              553.821929      796
796         523              542.724143      797
797         222              537.116317      798
798         501              531.153999      799
799         686              514.613124      800

[800 rows x 3 columns]


In [27]:
# -------------------------------
# 26. Calcular el promedio y desviación estándar de ventas por vendedor.
# -------------------------------
df26 = pd.read_sql("""
SELECT id_vendedor,
       AVG(precio_unitario * cantidad) AS promedio_venta_vendedor,
       STDEV(precio_unitario * cantidad) AS desv_por_vendedor
FROM ventas
GROUP BY id_vendedor;
""", engine)
print(df26)

    id_vendedor  promedio_venta_vendedor  desv_por_vendedor
0            23               760.129088         589.773116
1            46               795.559911         603.135172
2            29               769.719589         594.653452
3            15               789.537651         597.274796
4             9               785.134680         621.341561
5             3               755.285491         555.781734
6            32               794.376103         602.998834
7            26               785.834747         607.306618
8            12               767.950226         604.264536
9            35               770.455071         583.886721
10            6               776.429317         555.097890
11           43               780.986307         559.427874
12           49               793.870642         615.515763
13           27               795.532139         596.030240
14           21               801.639205         604.362998
15           38               813.636720

In [28]:
# -------------------------------
# 27. Detectar duplicados en registros de transacciones.
# -------------------------------
df27 = pd.read_sql("""
WITH t1 AS (
    SELECT id_venta, id_cliente, id_producto, fecha_venta, cantidad, precio_unitario, canal, region, estado_entrega,
           COUNT(*) AS Cant_identicos
    FROM ventas
    GROUP BY id_venta, id_cliente, id_producto, fecha_venta, cantidad, precio_unitario, canal, region, estado_entrega
)
SELECT * FROM t1 WHERE Cant_identicos > 1;
""", engine)
print(df27)

Empty DataFrame
Columns: [id_venta, id_cliente, id_producto, fecha_venta, cantidad, precio_unitario, canal, region, estado_entrega, Cant_identicos]
Index: []


In [30]:
# -------------------------------
# 28. Analizar clientes nuevos vs recurrentes en un periodo determinado.
# -------------------------------
df28 = pd.read_sql("""
WITH t1 AS (
    SELECT v.id_cliente, c.fecha_registro, v.fecha_venta,
           CASE WHEN c.fecha_registro <= v.fecha_venta THEN 'nuevo' ELSE 'antiguo' END AS NuevoAntiguo
    FROM ventas v INNER JOIN clientes c ON v.id_cliente = c.id_cliente
)
SELECT DATETRUNC(month, fecha_registro) AS periodo_registro,
       DATETRUNC(month, fecha_venta) AS periodo_venta,
       NuevoAntiguo,
       COUNT(NuevoAntiguo) AS cantidad
FROM t1
GROUP BY DATETRUNC(month, fecha_registro), DATETRUNC(month, fecha_venta), NuevoAntiguo;
""", engine)
print(df28)

    periodo_registro periodo_venta NuevoAntiguo  cantidad
0         2022-03-01    2024-12-01        nuevo        10
1         2021-06-01    2024-12-01        nuevo        37
2         2021-05-01    2024-07-01        nuevo        40
3         2020-05-01    2024-04-01        nuevo        42
4         2021-08-01    2025-05-01        nuevo        34
..               ...           ...          ...       ...
832       2020-04-01    2025-02-01        nuevo        39
833       2022-02-01    2024-10-01        nuevo        37
834       2021-05-01    2024-01-01        nuevo        41
835       2021-12-01    2023-05-01        nuevo        32
836       2020-02-01    2023-03-01        nuevo        42

[837 rows x 4 columns]


In [31]:
# -------------------------------
# 29. Calcular el crecimiento relativo del top 10 de clientes.
# -------------------------------
df29 = pd.read_sql("""
WITH t1 AS (
    SELECT TOP(10) id_cliente, SUM(precio_unitario * cantidad) AS venta_cliente
    FROM ventas
    GROUP BY id_cliente
)
SELECT a.id_cliente,
       DATETRUNC(month, v.fecha_venta) AS periodo,
       SUM(v.precio_unitario * v.cantidad) AS venta_mensual,
       LAG(SUM(precio_unitario * cantidad),1) OVER (PARTITION BY a.id_cliente ORDER BY DATETRUNC(month, v.fecha_venta)) AS venta_mes_anterior
FROM t1 a INNER JOIN ventas v ON a.id_cliente = v.id_cliente
GROUP BY a.id_cliente, DATETRUNC(month, v.fecha_venta);
""", engine)
print(df29)

     id_cliente     periodo  venta_mensual  venta_mes_anterior
0           142  2023-01-01    2439.079918                 NaN
1           142  2023-04-01     210.080002         2439.079918
2           142  2023-05-01     472.989990          210.080002
3           142  2023-06-01     628.700006          472.989990
4           142  2023-07-01    3128.570072          628.700006
..          ...         ...            ...                 ...
227         702  2024-12-01    3901.360077         1369.709930
228         702  2025-01-01     733.019989         3901.360077
229         702  2025-03-01    1269.159988          733.019989
230         702  2025-04-01    1770.860016         1269.159988
231         702  2025-05-01    2887.030029         1770.860016

[232 rows x 4 columns]


In [32]:
# -------------------------------
# 30. Crear un resumen mensual con ventas acumuladas, ranking y porcentaje sobre total.
# -------------------------------
df30 = pd.read_sql("""
SELECT DATEPART(month, fecha_venta) AS periodo,
       SUM(precio_unitario * cantidad) AS ventas_mes,
       SUM(SUM(precio_unitario * cantidad)) OVER (ORDER BY DATEPART(month, fecha_venta)) AS incremento_mensual,
       DENSE_RANK() OVER (ORDER BY SUM(precio_unitario * cantidad) DESC) AS ranking_de_ventas_mensuales,
       SUM(SUM(precio_unitario * cantidad)) OVER () AS total_venta,
       SUM(precio_unitario * cantidad) * 1.0 / SUM(SUM(precio_unitario * cantidad)) OVER () * 100 AS porcentaje
FROM ventas
GROUP BY DATEPART(month, fecha_venta);
""", engine)
print(df30)

    periodo    ventas_mes  incremento_mensual  ranking_de_ventas_mensuales  \
0         5  2.559070e+06        1.237904e+07                            1   
1         3  2.542465e+06        7.350359e+06                            2   
2         7  2.529583e+06        1.728170e+07                            3   
3         1  2.473469e+06        2.473469e+06                            4   
4         4  2.469610e+06        9.819969e+06                            5   
5         6  2.373077e+06        1.475212e+07                            6   
6         2  2.334426e+06        4.807894e+06                            7   
7        12  1.714663e+06        2.551963e+07                            8   
8        10  1.670676e+06        2.218971e+07                            9   
9         8  1.660317e+06        1.894202e+07                           10   
10       11  1.615261e+06        2.380497e+07                           11   
11        9  1.577018e+06        2.051903e+07                   